# 00 - Configurações e Funções Utilitárias
## Pipeline de Dados: Bronze → Silver → Gold

Este notebook contém:
- Configurações centralizadas (APIs, schemas, tabelas)
- Funções utilitárias reutilizáveis
- Constantes do projeto

## 1. Importação de Bibliotecas

In [ ]:
import requests
import json
from datetime import datetime, date
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *

## 2. Configurações de APIs

In [ ]:
# URLs das APIs
API_URLS = {
    'countries': 'https://restcountries.com/v3.1/independent?status=true',
    'exchange_rates': 'https://api.exchangerate-api.com/v4/latest/USD',
    'climate': 'https://api.open-meteo.com/v1/forecast'
}

# Configurações de request
REQUEST_CONFIG = {
    'timeout': 30,
    'max_retries': 3,
    'retry_delay': 5  # segundos
}

## 3. Configurações de Schemas e Tabelas

In [ ]:
# Schemas do Databricks
SCHEMAS = {
    'bronze': 'workspace.bronze',
    'silver': 'workspace.silver',
    'gold': 'workspace.gold'
}

# Tabelas Bronze
BRONZE_TABLES = {
    'countries': f"{SCHEMAS['bronze']}.countries_raw",
    'exchange_rates': f"{SCHEMAS['bronze']}.exchange_rates_raw"
}

# Tabelas Silver
SILVER_TABLES = {
    'dim_countries': f"{SCHEMAS['silver']}.dim_countries",
    'dim_currencies': f"{SCHEMAS['silver']}.dim_currencies",
    'dim_languages': f"{SCHEMAS['silver']}.dim_languages",
    'fact_country_metrics': f"{SCHEMAS['silver']}.fact_country_metrics"
}

# Tabelas Gold
GOLD_TABLES = {
    'countries_by_region': f"{SCHEMAS['gold']}.countries_by_region",
    'currency_usage': f"{SCHEMAS['gold']}.currency_usage",
    'language_distribution': f"{SCHEMAS['gold']}.language_distribution",
    'geographic_metrics': f"{SCHEMAS['gold']}.geographic_metrics"
}

## 4. Funções Utilitárias

In [ ]:
def fetch_api_data(url, timeout=30, max_retries=3):
    """
    Faz requisição HTTP com retry logic
    
    Args:
        url (str): URL da API
        timeout (int): Timeout em segundos
        max_retries (int): Número máximo de tentativas
    
    Returns:
        dict: Response JSON da API
    """
    import time
    
    for attempt in range(max_retries):
        try:
            response = requests.get(url, timeout=timeout)
            response.raise_for_status()
            return response.json()
        except requests.exceptions.RequestException as e:
            print(f"Tentativa {attempt + 1}/{max_retries} falhou: {str(e)}")
            if attempt < max_retries - 1:
                time.sleep(REQUEST_CONFIG['retry_delay'])
            else:
                raise Exception(f"Falha ao acessar API após {max_retries} tentativas: {str(e)}")

def get_execution_metadata():
    """
    Retorna metadados de execução
    
    Returns:
        dict: Timestamp e data de execução
    """
    now = datetime.now()
    return {
        'ingestion_timestamp': now,
        'execution_date': now.date()
    }

def log_metrics(stage, table_name, record_count, execution_time=None):
    """
    Loga métricas de execução
    
    Args:
        stage (str): Nome da camada (bronze/silver/gold)
        table_name (str): Nome da tabela
        record_count (int): Número de registros
        execution_time (float): Tempo de execução em segundos
    """
    print(f"\n{'='*60}")
    print(f"MÉTRICAS - {stage.upper()}")
    print(f"{'='*60}")
    print(f"Tabela: {table_name}")
    print(f"Registros: {record_count:,}")
    if execution_time:
        print(f"Tempo de execução: {execution_time:.2f} segundos")
    print(f"Timestamp: {datetime.now()}")
    print(f"{'='*60}\n")

def create_database_if_not_exists(spark, schema_name):
    """
    Cria database se não existir
    
    Args:
        spark: SparkSession
        schema_name (str): Nome do schema
    """
    spark.sql(f"CREATE DATABASE IF NOT EXISTS {schema_name}")
    print(f"Database '{schema_name}' verificado/criado com sucesso")

def optimize_table(spark, table_name, zorder_columns=None):
    """
    Otimiza tabela Delta com OPTIMIZE e Z-ORDER
    
    Args:
        spark: SparkSession
        table_name (str): Nome completo da tabela
        zorder_columns (list): Colunas para Z-ORDER
    """
    print(f"Otimizando tabela {table_name}...")
    
    if zorder_columns:
        zorder_cols = ', '.join(zorder_columns)
        spark.sql(f"OPTIMIZE {table_name} ZORDER BY ({zorder_cols})")
    else:
        spark.sql(f"OPTIMIZE {table_name}")
    
    print(f"Tabela {table_name} otimizada com sucesso")

## 5. Inicialização dos Schemas

In [ ]:
# Criar schemas se não existirem
def initialize_schemas(spark):
    """
    Inicializa todos os schemas necessários
    """
    for schema in SCHEMAS.values():
        create_database_if_not_exists(spark, schema)
    print("\nTodos os schemas foram inicializados!")

# Descomente para executar a inicialização
# initialize_schemas(spark)

## 6. Teste de Conectividade das APIs

In [ ]:
def test_api_connectivity():
    """
    Testa conectividade com todas as APIs configuradas
    """
    print("Testando conectividade das APIs...\n")
    
    for api_name, url in API_URLS.items():
        try:
            response = requests.get(url, timeout=10)
            status = "✓ OK" if response.status_code == 200 else f"✗ ERRO ({response.status_code})"
            print(f"{api_name:20s}: {status}")
        except Exception as e:
            print(f"{api_name:20s}: ✗ FALHA - {str(e)}")
    
    print("\nTeste de conectividade finalizado!")

# Descomente para executar o teste
# test_api_connectivity()

## 7. Configurações de Display

In [ ]:
# Configurações do Spark (compatível com Databricks Runtime 4.0.0)
# Habilitar Adaptive Query Execution se disponível
try:
    spark.conf.set("spark.sql.adaptive.enabled", True)
    print("✓ Adaptive Query Execution habilitado")
except:
    print("⚠ Adaptive Query Execution não disponível nesta versão")

print("\nConfigurações carregadas com sucesso!")
print(f"\nSchemas configurados:")
for key, value in SCHEMAS.items():
    print(f"  - {key}: {value}")